In [1]:
%matplotlib inline

import sys
sys.path.append('../')

import bisect
from collections import defaultdict
import datetime
import MySQLdb

from webhealth.analysis import AnalysisHelper

db_name='webhealth'
username='root'
password=''

db = MySQLdb.connect(host='localhost',
                         user=username,
                         passwd=password,
                         db=db_name)

ah = AnalysisHelper(username, password, db_name)

# utility functions

def get_node_ids():
    c = db.cursor()
    c.execute('select distinct node_id from metrics')    
    node_ids = [c.fetchone()[0] for _ in range(c.rowcount)]
    c.close()
    return node_ids


def plot_duration_and_success(node_id, website):
    c = db.cursor()

    c.execute('select reason, duration from metrics where website=%s', (website,))

    success_data = []
    duration_data = []
    reason_count = defaultdict(int)
    for _ in range(c.rowcount):
        reason, duration = c.fetchone()
        success = int(reason) == 0

        success_data.append(int(success))
        duration_data.append(float(duration))
        reason_count[reason] = reason_count[reason] + 1
    c.close()

    print reason_count
    
    import pandas as pd

    df = pd.DataFrame({
            'success': pd.Series(success_data),
            'duration': pd.Series(duration_data)
        })
    return df.plot(secondary_y='success', figsize=(16, 12))


def get_failed_websites(node_id, threshold_secs=121):
    """Find websites which have experienced two or more consecutive failures
    """
    probes_count = ah.get_probes_count(node_id)
    
    c = db.cursor()
    c.execute('select website, end_time from metrics where reason != 0 and node_id = %s order by end_time', (node_id,))
    
    failed_websites = set()
    website2failure_time = defaultdict(list)
    for _ in range(c.rowcount):
        website, end_time = c.fetchone()
        
        if not website2failure_time[website]:
            website2failure_time[website].append(end_time)
        else:
            if end_time <= website2failure_time[website][-1] + datetime.timedelta(seconds=threshold_secs):

                failed_websites.add(website)
                
                #if website == 'kinox.to':
                #    print end_time, website2failure_time[website][-1]
            
            website2failure_time[website].append(end_time)
    
    for k, v in list(website2failure_time.iteritems()):
        # I don't think it makes sense to take seriously websites which fail 20%
        # of time
        if 1.0 * len(v) / probes_count > 0.2:
            failed_websites.remove(k)
    
    return {w, website2failure_time[w]) for w in failed_websites]


node_ids = get_node_ids()
print "Node ids: {}".format(node_ids)

# I know that there are two nodes so far
failed_websites_0 = get_failed_websites(node_ids[0])
print "Number of failed websites {}: {}".format(node_ids[0], len(failed_websites_0))
failed_websites_1 = get_failed_websites(node_ids[1])
print "Number of failed websites {}: {}".format(node_ids[1], len(failed_websites_1))

failed = ah.find_failure_intersection(failed_websites_0, failed_websites_1)
len(failed)


Node ids: ['8b18188b10a7', 'aa4f2d96e411']
Number of failed websites 8b18188b10a7: 311
Number of failed websites aa4f2d96e411: 272


AttributeError: 'list' object has no attribute 'iteritems'